In [ ]:
import torch
from IPython.display import clear_output
from transformers import BertTokenizer




# 記得我們是使用中文 BERT
model_version = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_version)
magic_threshold = 0.6

# 情境句子
text_a = "[CLS]等到潮水[MASK]了,就知道誰沒穿褲子"
text_b = "等到潮水[MASK]了"
text_c = "就知道誰沒穿褲子"
text_d = "中 國 人 多 數 是 外 來 人 口 ， [MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK] ， 無 論 何 時 ， 中 醫 師 仍 然 很 難 相 信 這 是 病 理 徵 象 。 至 於 為 何 會 使 用 藥 品 ? 一 大 多 数 外 來 人 口 都 不 相 信 藥 品 ， 更 有 一 些 外 國 人 在 一 些 醫 院 開 的 藥 ， 甚 至 還 有 內 地 特 別 的 醫 院 ， 大 家 都 會 信 服 。 所 以 外 來 人 员 到 底 是 不 是 病 理 徵 象 ? 這 就 很 難 說 了 。 我 不 是 專 家 ， 但 我 想 在 此 跟 大 家 说 一 句 ： 中 医 是 中 国 药 ， 不 是 外 來 人 口 ！ 不 要 在 网 上 爆 太 多 病 例 ! 中 医 并 不 是 外 来 人 口 。 你 要 相 信 那 些 外 國 的 中 醫 是 世 界 性 的 。 你 肯 定 也 曾 在 网 上 发 现 有 中 醫 公 司 在 向 外 界 爆 料 。 但 是 一 切 以 真 相 为 依 据 ， 这 些 外 来 人 口 都 有 自 己 的 秘 密 。 我 不 相 信 中 医 是 某 一 个 有 一 个 神 奇 的 病 理 征 象 ， 只 知 道 中 医 能 改 变 很 多 人 的 生 理 结 构 ， 而 且 很 有 效 ， 因 为 它 不 需 要 太 多 的 医 学 治 疗 ， 所 以 真 正 有 效 根 本 不 存 在 。"
text_e = "中 國 人 多 數 是 外 來 人 口 ， 大 量 外 來 人 員 ， 不 是 內 地 客 人 ， 無 論 何 時 ， 中 醫 師 仍 然 很 難 相 信 這 是 病 理 徵 象 。 至 於 為 何 會 使 用 藥 品 ? 一 大 多 数 外 來 人 口 都 不 相 信 藥 品 ， 更 有 一 些 外 國 人 在 一 些 醫 院 開 的 藥 ， 甚 至 還 有 內 地 特 別 的 醫 院 ， 大 家 都 會 信 服 。 所 以 外 來 人 员 到 底 是 不 是 病 理 徵 象 ? 這 就 很 難 說 了 。 我 不 是 專 家 ， 但 我 想 在 此 跟 大 家 说 一 句 ： 中 医 是 中 国 药 ， 不 是 外 來 人 口 ！ 不 要 在 网 上 爆 太 多 病 例 ! 中 医 并 不 是 外 来 人 口 。 你 要 相 信 那 些 外 國 的 中 醫 是 世 界 性 的 。 你 肯 定 也 曾 在 网 上 发 现 有 中 醫 公 司 在 向 外 界 爆 料 。 但 是 一 切 以 真 相 为 依 据 ， 这 些 外 来 人 口 都 有 自 己 的 秘 密 。 我 不 相 信 中 医 是 某 一 个 有 一 个 神 奇 的 病 理 征 象 ， 只 知 道 中 医 能 改 变 很 多 人 的 生 理 结 构 ， 而 且 很 有 效 ， 因 为 它 不 需 要 太 多 的 医 学 治 疗 ， 所 以 真 正 有 效 根 本 不 存 在 。"
text_f = "印度面臨「海嘯式」的新冠疫情，確診及死亡人數節節上升。剛離開印度回國的中國人蒙姐近日就其所見所聞娓娓道來，她認為，專家預測印度實際感染人數要比公佈的數字多3至5倍應是真的，在印度，即使確診了也根本沒有溯源一說，沒有任何強制性核酸檢測，而據她觀察當地很多人對生死看得特別淡，心態與中國人大不同。"
#
def get_input_from_mask_sentence(a):
    tokens = tokenizer.tokenize(a)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    
    # 除了 tokens 以外我們還需要辨別句子的 segment ids
    input_ids = torch.tensor([ids])  # (1, seq_len)
    token_type_ids = torch.zeros_like(input_ids)  # (1, seq_len)
    return tokens, input_ids ,token_type_ids

#
def get_input_from_two_sentence(a, b):
    inputs = tokenizer.encode_plus(a, b, return_tensors='pt', add_special_tokens=True)
    input_ids ,token_type_ids = inputs['input_ids'],inputs['token_type_ids']
    
    return input_ids ,token_type_ids

In [ ]:
# 潤句1:一次偵測
# 適合用於微調語法
def convert2text(predictions,k=3):
    probs, indices = torch.topk(torch.softmax(predictions, -1), k)
    predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())
    return predicted_tokens,probs


from transformers import BertForMaskedLM

#
maskedLM_model = BertForMaskedLM.from_pretrained(model_version)
clear_output()

tokens, input_ids ,token_type_ids = get_input_from_mask_sentence(text_f)

# 使用 masked LM 估計 [MASK] 位置所代表的實際 token 
maskedLM_model.eval()
with torch.no_grad():
    outputs = maskedLM_model(input_ids, token_type_ids=token_type_ids)
    predictions = outputs[0]
    # (1, seq_len, num_hidden_units)
del maskedLM_model

k=3
for idx in range(0,len(tokens)):
    predicted_tokens,probs = convert2text(predictions[0,idx],k=k)
    boolresult= False
    for tmp_p,tmp_t in zip(probs,predicted_tokens):
        #
        if tokens[idx] == tmp_t:
            continue
        #
        if float(tmp_p) > magic_threshold or tokens[idx]=="[MASK]":
            print(tokens[idx],tmp_t,f" v ->{tmp_p}")
            boolresult = True
            break
    if boolresult is False:
        print(tokens[idx],tokens[idx])

In [ ]:
# 潤句2：逐字偵測
# 適合用於產生銜接句
def convert2text(predictions,k=1):
    probs, indices = torch.topk(torch.softmax(predictions, -1), k)
    predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())
    return predicted_tokens,probs


from transformers import BertForMaskedLM

#
maskedLM_model = BertForMaskedLM.from_pretrained(model_version)
clear_output()

tokens, input_ids ,token_type_ids = get_input_from_mask_sentence(text_d)
for idx,tmp_token in enumerate(tokens):
    tmp_text = tokens[:]
    tmp_text[idx]="[MASK]"
    tmp_tokens, tmp_input_ids ,tmp_token_type_ids = get_input_from_mask_sentence("".join(tmp_text))
    with torch.no_grad():
        outputs = maskedLM_model(tmp_input_ids, token_type_ids=tmp_token_type_ids)
        predictions = outputs[0]
        # (1, seq_len, num_hidden_units)
    pred_token,prob = convert2text(predictions[0,idx])
    if prob[0] > magic_threshold and tokens[idx]!=pred_token[0]:
        print(f"{tokens[idx]},{pred_token[0]} --> {prob}")
        tokens[idx] = str(pred_token[0])
    elif tokens[idx]=="[MASK]":
        print(f"{tokens[idx]},{pred_token[0]} --> {prob}")
        tokens[idx] = str(pred_token[0])
    else:
        print(f"{tokens[idx]},{tokens[idx]}")
del maskedLM_model